# Entrenamiento de una Red neuronal Pequeña.
Código Basado en https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [2]:
from math import exp
from random import seed, random
import math
import numpy as np

* Una red neuronal es una función universal con  3 tipos de capas: capa de entrada, capas intermedia capa de salida. 

* Un problema es que la derivada no se puede calcular directamente. Desde la la capa de salida, solo se tiene acceso a las derivadas parciales de la capa antecesora inmediata.

* Por lo tanto se requiere un algoritmo especial para calcular la derivada basada en la regla de la cadena de cálculo. 

* Esta indica que si se tienen funciones anidadas: $y=g(x), z = f(y)$ o bien $z = f(g(x))$, una manera de resolver esta anidación es aplicando la regla de la cadena. Ejemplo:

$$\frac{d z}{d x} = \frac{dz}{dy} \frac{dy}{dx} $$

* El algoritmo de **Propagación hacia atrás** (Backpropagation, BP), es un algoritmo de entrenamiento supervisado permite calcular la derivada de una red neuronal y por lo tanto actualizar los pesos de todas las capas intermedias.

* Por lo tanto describiremos un código de una red neuronal con fines didácticos. 




Ejemplo Red neuronal multicapa

**Inicializando la red**

initialize_network recibe el tamaño del vector de entrada, el número de capas ocultas y el numero de capas de salida.

Si inicializamos initialize_network(3, 2, 2), tendremos 2 conjuntos de 3+1 pesos considerando el *bias* para la primera capa que asocia cada entrada de $[x_0,x_1,x_0]$ con cada neurona intermedia. 


In [19]:

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network = initialize_network(2, 2, 1)

print("capa de entrada:", network[0])


capa de entrada: [{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}]


In [9]:
network[1]

[{'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}]

**Propagación hacia adelante** (Forwad propagation): salida de la red neuronal dada una entrada $\mathbf{x}$

Es la función que nos permite utilizar la red neuronal para hacer clasificación. La entrada se va procesando a través de las capas hasta obtener una salida.

**Activación de la neurona**

Es el cálculo de una regresión lineal de la forma 

$$y = \mathbf{x}\mathbf{w}^T + w_0 $$

In [10]:

#calcular activacion de la neurona por una entrada
def activate(weights, inputs):
    # W*X+w_0
    
    #bias: w_0
    activation = weights[-1]
    
    #calcular w_0 + W*X
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
        
    return activation

**Transferencia de la neurona**

Existen muchas funciones de transferencia como relu, tanh identidad, pero una de las mas utilizadas es la función sigmoide con forma de s o también llamada curva logística. Tiene la forma:

$$\hat{z} = \text{sigm}(y) = 1/(1+e^{-y}))$$

In [11]:
# Neurona
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

**Derivada de transferencia hacia atrás de las capas intermedias**

Esta derivada es solo una parte de la derivda de toda la red neuronal que se hace en forma encadenada. 

$$ \frac{\partial \hat{z}}{\partial y} = \hat{z} (1-\hat{z} )   $$

In [12]:
# Esta es la derivada de la función sigmoide
def transfer_derivative(output):
    return output * (1.0 - output)

**Propagación hacia adelante**

Es la red neuronal en funcionamiento. Recibe una entrada $\mathbf{x_i}$ y regresa una salida $y_i$

In [21]:
# Propagación hacia adelante
def forward_propagate(network, inputs):

    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [28]:
forward_propagate(network,[1,4])

[0.8138219506313304]

In [29]:
network[1]

[{'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349],
  'output': 0.8138219506313304}]

**Propagación hacia atrás del error**



In [32]:

 
# Propaga el error hacia atrás y lo va guardando en la estructura de la red
def backward_propagate_error(network, expected):
    
    #comenzamos con la capa de salida
    for i in reversed(range(len(network))):
        
        # por cada capa de la red asignada a layer
        layer = network[i]
        
        #el tamaño de la capa es equivalente al numero de neuronas en la capa
        n_neuronas = len(layer)
        
        #inicializamos error
        errors = list()
        
        # la primera vez calculamos sobre la ultima capa que es a la que tenemos acceso en un inicio
        if i == len(network)-1:
            
            # por cada neurona de la capa hacer:
            for j in range(n_neuronas):
            
                neuron = layer[j]

                yhat = neuron['output']
                y = expected[j]
            
                #esta es la derivada de la función de costo (o error) basada en la log verosimilitud
                # Aqui comienza la transferencia
                derivada_error = (yhat-y)/((yhat-1)*yhat)
                errors.append(derivada_error) 
        
        else:
            #aqui son las capas restantes hacia atrás donde propagamos el error
            
            for j in range(n_neuronas):
                error = 0.0
                for neuron in network[i + 1]:
                    
                    #son los pesos de las entradas de la ultima capa
                    error += (neuron['weights'][j] * neuron['delta'])
                    
                errors.append(error)
                
        #aqui guardamos todos los delta asociada a cada neurona de cada capa
        for j in range(n_neuronas):
            neuron = layer[j]
            
            #el error magnifica la derivada
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 

In [35]:
backward_propagate_error(network,[1])

In [34]:
network

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614],
   'output': 0.9864519743186091,
   'delta': 0.0016212755200530635},
  {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381],
   'output': 0.9362134079958969,
   'delta': 0.008769148617603043}],
 [{'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349],
   'output': 0.8138219506313304,
   'delta': 0.18617804936866958}]]

**Actualizar pesos con el error**

Se recibe la red con los deltas, y un vector de entrada $row$ y la tasa de aprendizaje $l_rate$ asignada por el usuario la cual especifica en porcentaje de actualización que queremos para los pesos.

In [18]:
# Actualizar los pesos de la red 
def update_weights(network, row, l_rate):
    
    # por cada capa de la red
    for i in range(len(network)):
        
        #extraer la clasificación real del vector de entrada. 
        inputs = row[:-1]
        
        #las salidas ahora son las entradas de la siguiente capa
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
 
        layer = network[i]
        for neuron in layer:
            for j in range(len(inputs)):
                
                # por cada neurona y cada una de sus entradas
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            
            #actualización del bias
            neuron['weights'][-1] += l_rate * neuron['delta']
           

       
 

**Entrenar la red con Gradiente Descendiente estocástico**

* una característica diferente a Gradiente descendiente normal, es que por cada iteación, utiliza un vector de entrada diferente para actualizar los pesos.

In [37]:


def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        
        #por cada vector en el conjunto de entrenamiento.
        for row in train:
            
            # se calcular la salida de un elemento del vector
            outputs = forward_propagate(network, row)
            
            #preparamos con ceros
            expected = [0 for i in range(n_outputs)]
            
            #la posición row[-1] \in {0,1} tendrá un 1
            expected[int(row[-1])] = 1
            
            #sumamos cada error en cada dimensión diferente
            
       #     sum_error += sum([(expected[i]-outputs[i] )**2 for i in range(len(expected))])
            
            zi = expected[1]
            zi_e = outputs[1]
            
            #aquí el error reportado es la suma log verosimilitud , queremos que sea el máximo
            #sum_error += zi*math.log(zi_e)+(1-zi)*math.log(1-zi_e)
            sum_error += zi*math.log(zi_e)+(1-zi)*math.log(1-zi_e)
        
      
            #propagamos hacia atrás 
            #el error generado por la diferencia de el valor esperado y el valor generado por la red
            # se reajusta la red para predecir mejor el valor esperado
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
 



In [38]:


dataset = [[ 0.        , 43.57879291,  0.        ],
       [ 1.        , 38.25686159,  0.        ],
       [ 2.        , 33.65222425,  0.        ],
       [ 3.        , 35.58326609,  0.        ],
       [ 4.        , 34.71588722,  0.        ],
       [ 5.        , 32.38702072,  0.        ],
       [ 6.        , 28.39419039,  0.        ],
       [ 7.        , 20.20082333,  0.        ],
       [ 8.        , 24.19586975,  0.        ],
       [ 9.        , 18.48536939,  0.        ],
       [10.        , 19.43098795,  0.        ],
       [11.        ,  9.78931084,  0.        ],
       [12.        , 14.4403072 ,  0.        ],
       [13.        ,  8.95888303,  0.        ],
       [14.        ,  7.17704765,  0.        ],
       [15.        ,  9.57779397,  0.        ],
       [16.        ,  7.58424501,  0.        ],
       [17.        ,  8.9341588 ,  0.        ],
       [18.        ,  3.13378008,  0.        ],
       [19.        ,  9.23532406,  0.        ],
       [20.        ,  6.43370902,  0.        ],
       [21.        ,  6.67961846,  0.        ],
       [22.        ,  5.15107388,  0.        ],
       [23.        ,  3.13252828,  0.        ],
       [24.        ,  9.40036651,  0.        ],
       [25.        ,  8.96646931,  0.        ],
       [26.        , 12.97187713,  0.        ],
       [27.        ,  6.17097537,  0.        ],
       [28.        , 14.94380716,  0.        ],
       [29.        , 15.78038035,  0.        ],
       [30.        , 13.44006058,  0.        ],
       [31.        , 12.38640541,  0.        ],
       [32.        , 18.24681531,  0.        ],
       [33.        , 24.89568658,  0.        ],
       [34.        , 20.9740082 ,  0.        ],
       [35.        , 29.60592301,  0.        ],
       [36.        , 28.15828701,  0.        ],
       [37.        , 36.2793585 ,  0.        ],
       [38.        , 35.14373666,  0.        ],
       [39.        , 45.91909167,  0.        ],
       [ 0.        , 72.99963497,  1.        ],
       [ 1.        , 66.25249531,  1.        ],
       [ 2.        , 69.47862325,  1.        ],
       [ 3.        , 68.12423544,  1.        ],
       [ 4.        , 58.18406218,  1.        ],
       [ 5.        , 59.71572434,  1.        ],
       [ 6.        , 56.45457802,  1.        ],
       [ 7.        , 52.4068255 ,  1.        ],
       [ 8.        , 47.24274882,  1.        ],
       [ 9.        , 43.36311311,  1.        ],
       [10.        , 44.07483255,  1.        ],
       [11.        , 39.85302512,  1.        ],
       [12.        , 44.01192218,  1.        ],
       [13.        , 41.07318483,  1.        ],
       [14.        , 33.8127034 ,  1.        ],
       [15.        , 33.21045858,  1.        ],
       [16.        , 37.15741812,  1.        ],
       [17.        , 31.73730861,  1.        ],
       [18.        , 34.83305033,  1.        ],
       [19.        , 39.87419754,  1.        ],
       [20.        , 31.48844934,  1.        ],
       [21.        , 35.0882841 ,  1.        ],
       [22.        , 39.71262275,  1.        ],
       [23.        , 31.62777132,  1.        ],
       [24.        , 40.68066481,  1.        ],
       [25.        , 36.14619892,  1.        ],
       [26.        , 38.39007043,  1.        ],
       [27.        , 35.30625817,  1.        ],
       [28.        , 38.9855814 ,  1.        ],
       [29.        , 44.46375169,  1.        ],
       [30.        , 42.9567545 ,  1.        ],
       [31.        , 43.13876601,  1.        ],
       [32.        , 46.36785753,  1.        ],
       [33.        , 53.20209487,  1.        ],
       [34.        , 54.57671019,  1.        ],
       [35.        , 61.72180758,  1.        ],
       [36.        , 56.20328364,  1.        ],
       [37.        , 60.75140654,  1.        ],
       [38.        , 62.61155513,  1.        ],
       [39.        , 67.27858905,  1.        ]]

In [39]:
dataset = np.array(dataset)

dataset[:,0] = dataset[:,0]/max(dataset[:,0])
dataset[:,1] = dataset[:,1]/max(dataset[:,1])
dataset

array([[0.        , 0.59697275, 0.        ],
       [0.02564103, 0.52406922, 0.        ],
       [0.05128205, 0.46099168, 0.        ],
       [0.07692308, 0.48744444, 0.        ],
       [0.1025641 , 0.47556248, 0.        ],
       [0.12820513, 0.44366004, 0.        ],
       [0.15384615, 0.38896346, 0.        ],
       [0.17948718, 0.27672499, 0.        ],
       [0.20512821, 0.33145193, 0.        ],
       [0.23076923, 0.2532255 , 0.        ],
       [0.25641026, 0.26617925, 0.        ],
       [0.28205128, 0.13410082, 0.        ],
       [0.30769231, 0.19781342, 0.        ],
       [0.33333333, 0.12272504, 0.        ],
       [0.35897436, 0.09831621, 0.        ],
       [0.38461538, 0.13120331, 0.        ],
       [0.41025641, 0.10389429, 0.        ],
       [0.43589744, 0.12238635, 0.        ],
       [0.46153846, 0.04292871, 0.        ],
       [0.48717949, 0.12651192, 0.        ],
       [0.51282051, 0.08813344, 0.        ],
       [0.53846154, 0.09150208, 0.        ],
       [0.

In [41]:
n_inputs = len(dataset[0]) - 1
print("inputs: ", n_inputs)

#total possible number of output values
n_outputs = len(set([ int(row[-1]) for row in dataset]))
print("outputs: ", n_outputs)

network = initialize_network(n_inputs, 4, n_outputs)
train_network(network, dataset, 0.01, 10000, n_outputs)
for layer in network:
    print(layer)

inputs:  2
outputs:  2


NameError: name 'update_weights' is not defined

In [24]:
e = 0
for x in dataset:
    
    y = forward_propagate(network, x[:2])
    print("expectation:",x[-1],"reality",1*((y[-1])>0.5))
    
    e+= (x[-1]- 1*((y[-1])>0.5))**2
    

expectation: 0.0 reality 1
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 0
expectation: 0.0 reality 1
expectation: 0.0 reality 0
expectation: 0.0 reality 1
expectation: 0.0 reality 1
e

In [16]:
e

10.0

In [15]:
j=0
i=0

n_neuronas = len(network[i])

for j in range(n_neuronas):
    print("neurna no.",j)
    error = 0

    for neuron in network[i+1]:
   #     print("layer:",neuron)
   #     print("weights",i+1,j,neuron["weights"][j])
        error += neuron["weights"][j]*neuron['delta']
    print(error)

neurna no. 0
-0.524819613827942
neurna no. 1
-0.06621286561628859
neurna no. 2
-0.4060077630258488
neurna no. 3
2.1053948455209897
